In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# Load CIFAR-100 data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Preprocess the data
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train, 100), to_categorical(y_test, 100)

# Load VGG16 model with pre-trained weights, excluding top layers
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the layers except the last 4 layers
for layer in vgg16.layers[:-4]:
    layer.trainable = False

# Create a new model
model = models.Sequential()
model.add(vgg16)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(100, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

# Save the model
model.save('cifar10_vgg16.h5')


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Load the trained model
model = load_model('cifar10_vgg16.h5')

# CIFAR-10 class labels
cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Function to preprocess and predict a single image
def preprocess_and_predict(image_path, model, class_labels):
    # Load the image
    img = load_img(image_path, target_size=(32, 32))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0) / 255.0

    # Predict using the model
    prediction = model.predict(img)
    class_idx = np.argmax(prediction[0])
    class_name = class_labels[class_idx]

    return class_name

# Predict from CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_test = x_test / 255.0
cifar10_img = np.expand_dims(x_test[0], axis=0)
prediction = model.predict(cifar10_img)
class_idx = np.argmax(prediction[0])
class_name = cifar10_classes[class_idx]
print(f'Prediction for CIFAR-10 image: {class_name}')

 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to convert a tensor to a numpy array
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    #npimg = img.numpy()
    plt.imshow(img)
    plt.show()

imshow(x_test[0])